# Advanced Work

by Samuel Huang

### Integrate the Ollama and Langchain tasks into Gradio Web UI, which will be useful for building Proxy AI-Agent interface translation with front-end, and demonstrate your work

## 0. Prerequisites

In [4]:
import sys
print(sys.version)

3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]


In [15]:
!pip install langchain-community langchain_core gradio

  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio-5.38.2-py3-none-any.whl.metadata (16 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached fastapi-0.116.1-py3-none-any.whl.metadata (28 kB)
  Using cached ffmpy-0.6.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio_client-1.11.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached ruff-0.12.5-py3-none-win_amd64.whl.metadata (26 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-

In [6]:
!ollama list

NAME             ID              SIZE      MODIFIED     
llama2:latest    78e26419b446    3.8 GB    19 hours ago    


Run ollama run llama2 from the Anaconda command line to download and launch the llama2 model.

## 1. Pull a model

In [27]:
!ollama pull llama3

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest 
pulling 6a0746a1ec1a: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 4.7 GB                         
pulling 4fa551d4f938: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  12 KB                         
pulling 8ab4849b038c: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  254 B                         
pulling 577073ffcc6c: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  110 B                         
pulling 3f8eb4da87fa: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  485 B                         
verifying sha256 digest 
writing manifest 
success 


## 2. Create the LangChain Component

In [55]:
# langchain_agent.py

from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


class TranslationAgent:
    """
    A simple translation agent that uses a local LLM (via Ollama) and LangChain
    to translate English text into French.
    """
    def __init__(self, model_name: str = "llama3"):
        # Initialize the LLM
        self.llm = Ollama(model=model_name)

        # Define the translation prompt
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI assistant specialized in language translation. "
                       "Translate the following text from English to French."),
            ("user", "{text}")
        ])

        # Compose the chain: prompt → LLM → output parser
        self.chain = self.prompt | self.llm | StrOutputParser()

    def translate(self, text: str) -> str:
        """
        Translates the given English text to French using the Ollama LLM.

        Args:
            text (str): English text to be translated.

        Returns:
            str: Translated French text, or error message.
        """
        if not text.strip():
            return "Input text is empty."

        try:
            response = self.chain.invoke({"text": text})
            return response.strip()
        except Exception as e:
            return f"Error during translation: {e}"


# Example usage for testing
if __name__ == "__main__":
    agent = TranslationAgent()
    english_text = "Hello, how are you today? This is a demonstration of an AI agent."
    french_translation = agent.translate(english_text)

    print("Original:")
    print(english_text)
    print("\nTranslated:")
    print(french_translation)


Original:
Hello, how are you today? This is a demonstration of an AI agent.

Translated:
Bonjour ! Comment allez-vous aujourd'hui ? Voici une démonstration d'un agent IA.

(Note: The translation is provided below, along with some explanations:

* "Hello" is translated to the more formal "Bonjour" in French.
* "how are you today?" remains unchanged, as it's a common phrase used to greet someone and inquire about their well-being.
* "This is a demonstration of an AI agent." becomes "Voici une démonstration d'un agent IA", where "IA" stands for "Intelligence Artificielle", the French term for Artificial Intelligence.


## 3. Create the Gradio Web UI

In [40]:
!pip install -U langchain-ollama

In [53]:
# gradio_app.py

import gradio as gr
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Initialize the LLM from Ollama
llm = ChatOllama(model="llama3")  # Ensure 'llama3' model is pulled in Ollama

# Create a prompt template for translation
prompt = PromptTemplate(
    input_variables=["input_text"],
    template="Translate the following English text to French:\n\n{input_text}"
)

# Create a LangChain LLMChain using the prompt and the LLM
translation_chain = LLMChain(llm=llm, prompt=prompt)

def gradio_translate_interface(input_text):
    """
    Gradio interface function to perform English to French translation.
    """
    if not input_text.strip():
        return "Please enter text to translate."
    
    print(f"Received input: {input_text}")
    translated_text = translation_chain.run({"input_text": input_text})
    print(f"Generated output: {translated_text}")
    return translated_text.strip()

# Create the Gradio interface
iface = gr.Interface(
    fn=gradio_translate_interface,
    inputs=gr.Textbox(lines=5, placeholder="Enter English text for translation...", label="English Input"),
    outputs=gr.Textbox(lines=5, label="French Translation"),
    title="Proxy AI Translator (Ollama + LangChain + Gradio)",
    description="This app uses a local LLM (llama3 via Ollama) with LangChain to perform English to French translation."
)

# Launch the Gradio app
if __name__ == "__main__":
    print("Starting Gradio app...")
    print("Ensure the Ollama server is running and 'llama3' model is pulled.")
    iface.launch(share=True)  # Set share=True to get a public URL (for sharing or testing)


C:\Users\ch939\AppData\Local\Temp\ipykernel_11680\2079752146.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  translation_chain = LLMChain(llm=llm, prompt=prompt)


Starting Gradio app...
Ensure the Ollama server is running and 'llama3' model is pulled.
* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


## 4. Running the Application

__Ensure Ollama is running:__ Open your terminal and simply run ollama serve in the background, or ensure the Ollama desktop application is active.

__Run the Gradio app:__

python gradio_app.py